In [192]:
import pandas as pd
import numpy as np

In [193]:
circuits = pd.read_csv('../data/circuits.csv')
constructor_results = pd.read_csv('../data/constructor_results.csv')
constructor_standings = pd.read_csv('../data/constructor_standings.csv')
constructors = pd.read_csv('../data/constructors.csv')
driver_standings = pd.read_csv('../data/driver_standings.csv')
drivers = pd.read_csv('../data/drivers.csv', parse_dates=[6])
lap_times = pd.read_csv('../data/lap_times.csv')
pit_stops = pd.read_csv('../data/pit_stops.csv')
qualifying = pd.read_csv('../data/qualifying.csv')
races = pd.read_csv('../data/races.csv', parse_dates=[5])
results = pd.read_csv('../data/results.csv')
seasons = pd.read_csv('../data/seasons.csv')
status = pd.read_csv('../data/status.csv')

In [194]:
result_with_dob = results.merge(drivers, left_on='driverId', right_on='driverId')
results_with_dob_date = result_with_dob.merge(races, on = 'raceId', how = 'left')

In [195]:
#results_with_dob_date['ageAtRace'] = pd.to_datetime(results_with_dob_date['date']) - pd.to_datetime(results_with_dob_date['dob'])
pd.to_datetime(results_with_dob_date['date'])
pd.to_datetime(results_with_dob_date['dob'])

0       1985-01-07
1       1985-01-07
2       1985-01-07
3       1985-01-07
4       1985-01-07
           ...    
24955   1995-06-29
24956   1995-06-29
24957   1995-09-23
24958   1996-06-25
24959   1996-06-25
Name: dob, Length: 24960, dtype: datetime64[ns]

In [196]:
#Calculate age at race in days (timedelta)
results_with_dob_date['ageAtRace'] = results_with_dob_date['date'] - results_with_dob_date['dob']

In [197]:
#Calculate how many days in f1 in days (timedelta)
date_of_debut = results_with_dob_date.groupby('driverId')['date'].min().to_frame()
date_of_debut = date_of_debut.merge(drivers, on = 'driverId', how = 'left').rename(columns={'date':'date_of_debut'})
date_of_debut = date_of_debut[['driverId','date_of_debut']]
date_of_debut['date_of_debut']
df = results_with_dob_date.merge(date_of_debut, on = 'driverId', how = 'left')
df['ageAtDebut'] = df['date_of_debut'] - df['dob']
df['yearsExperience'] = df['date'] - df['date_of_debut']


In [198]:
#Drop columns
df = df.drop(columns=['number_y', 'number_x', 'code', 'forename', 'surname', 'dob', 'url_x', 'time_y', 'url_y'])

In [199]:
#Rename Columns
df = df.rename(columns={"grid": "gridStart", "name": "circuitName", "date_of_debut" : "dateOfDebut", "points": "pointsGained"})

In [200]:
#Add is racing at home feature
df['racingAtHome'] = df.apply(lambda row: row.nationality in row.circuitName, axis=1)

In [201]:
#COnvert ages from days to years
df['ageAtRace'] = df.apply(lambda row: row.ageAtRace / pd.Timedelta('365 days'), axis=1)
df['ageAtDebut'] = df.apply(lambda row: row.ageAtDebut / pd.Timedelta('365 days'), axis=1)
df['yearsExperience'] = df.apply(lambda row: row.yearsExperience / pd.Timedelta('365 days'), axis=1)

In [202]:
#Add starts in front 2 places on the grid
df['startsFrontRow'] = df['gridStart'] <= 2

In [203]:
#Merge with driver standings
driver_standings = driver_standings.rename(columns={"points": "driverStandingsPoints", "position": "driverStandingsPosition", "wins" : "driverStandingsWins"})
driver_standings = driver_standings.drop(columns=['positionText'])
df = df.merge(driver_standings, on = ['raceId','driverId'], how = 'left')

In [204]:
#Get driver standings information before race
df['driverStandingsPoints'] = df.groupby(['year','driverId'])['driverStandingsPoints'].shift(fill_value=0)
df['driverStandingsPosition'] = df.groupby(['year','driverId'])['driverStandingsPosition'].shift(fill_value=0)
df['driverStandingsWins'] = df.groupby(['year','driverId'])['driverStandingsWins'].shift(fill_value=0)

In [205]:
df

,resultId,raceId,driverId,constructorId,gridStart,position,positionText,positionOrder,pointsGained,laps,...,ageAtRace,dateOfDebut,ageAtDebut,yearsExperience,racingAtHome,startsFrontRow,driverStandingsId,driverStandingsPoints,driverStandingsPosition,driverStandingsWins
0,1,18,1,1,1,1,1,1,10.0,58,...,23.202740,2007-03-18,22.205479,0.997260,False,True,1.0,0.0,0.0,0.0
1,27,19,1,1,9,5,5,5,4.0,56,...,23.221918,2007-03-18,22.205479,1.016438,False,False,9.0,10.0,1.0,1.0
2,57,20,1,1,3,13,13,13,0.0,56,...,23.260274,2007-03-18,22.205479,1.054795,False,False,27.0,14.0,1.0,1.0
3,69,21,1,1,5,3,3,3,6.0,66,...,23.317808,2007-03-18,22.205479,1.112329,False,False,48.0,14.0,3.0,1.0
4,90,22,1,1,3,2,2,2,8.0,58,...,23.356164,2007-03-18,22.205479,1.150685,False,False,69.0,20.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24955,24943,1046,849,3,16,\N,R,18,0.0,52,...,25.457534,2020-07-05,25.035616,0.421918,False,False,70151.0,0.0,20.0,0.0
24956,24962,1047,849,3,18,17,17,17,0.0,54,...,25.476712,2020-07-05,25.035616,0.441096,False,False,70197.0,0.0,21.0,0.0
24957,24941,1046,851,3,17,16,16,16,0.0,87,...,25.221918,2020-12-06,25.221918,0.000000,False,False,70162.0,0.0,0.0,0.0
24958,24942,1046,850,210,20,17,17,17,0.0,87,...,24.465753,2020-12-06,24.465753,0.000000,False,False,70163.0,0.0,0.0,0.0


In [206]:
#Merge with constructor standings
constructor_standings = constructor_standings.rename(columns={"points": "constructorStandingsPoints", "position": "constructorStandingsPosition", "wins" : "constructorStandingsWins"})
constructor_standings = constructor_standings.drop(columns=['positionText'])
df = df.merge(constructor_standings, on = ['raceId','constructorId'], how = 'left')

In [207]:
#Get constructor standings information before race
df['constructorStandingsPoints'] = df.groupby(['year','driverId'])['constructorStandingsPoints'].shift(fill_value=0)
df['constructorStandingsPosition'] = df.groupby(['year','driverId'])['constructorStandingsPosition'].shift(fill_value=0)
df['constructorStandingsWins'] = df.groupby(['year','driverId'])['constructorStandingsWins'].shift(fill_value=0)

In [208]:
#races_won_by_driver = df.groupby('driverId')['positionOrder'].apply(lambda x: (x==1).sum())
df.date.unique()

array(['2008-03-16T00:00:00.000000000', '2008-03-23T00:00:00.000000000',
       '2008-04-06T00:00:00.000000000', ...,
       '1950-05-30T00:00:00.000000000', '1950-05-13T00:00:00.000000000',
       '1950-06-18T00:00:00.000000000'], dtype='datetime64[ns]')

In [209]:
i = 0
length = len(df.date.unique())
for date in df.date.unique():
    print(i,' em ',length)
    i+=1
    races_won_by_driver = df[df.date < date].groupby('driverId')['positionOrder'].apply(lambda x: (x==1).sum()).reset_index(name='racesWon')
    races_won_by_driver['date'] = date
    dfaux = df[df['date'] == date].copy()
    dfaux['racesWon'] = 'to_delete'
    dfaux = dfaux.drop('racesWon', axis = 1)
    df.loc[df['date'] == date, 'racesWon'] = dfaux.merge(races_won_by_driver, on=['date','driverId'], how='left')['racesWon'].values


0  em  1035
1  em  1035
2  em  1035
3  em  1035
4  em  1035
5  em  1035
6  em  1035
7  em  1035
8  em  1035
9  em  1035
10  em  1035
11  em  1035
12  em  1035
13  em  1035
14  em  1035
15  em  1035
16  em  1035
17  em  1035
18  em  1035
19  em  1035
20  em  1035
21  em  1035
22  em  1035
23  em  1035
24  em  1035
25  em  1035
26  em  1035
27  em  1035
28  em  1035
29  em  1035
30  em  1035
31  em  1035
32  em  1035
33  em  1035
34  em  1035
35  em  1035
36  em  1035
37  em  1035
38  em  1035
39  em  1035
40  em  1035
41  em  1035
42  em  1035
43  em  1035
44  em  1035
45  em  1035
46  em  1035
47  em  1035
48  em  1035
49  em  1035
50  em  1035
51  em  1035
52  em  1035
53  em  1035
54  em  1035
55  em  1035
56  em  1035
57  em  1035
58  em  1035
59  em  1035
60  em  1035
61  em  1035
62  em  1035
63  em  1035
64  em  1035
65  em  1035
66  em  1035
67  em  1035
68  em  1035
69  em  1035
70  em  1035
71  em  1035
72  em  1035
73  em  1035
74  em  1035
75  em  1035
76  em  1035
77  em  1

594  em  1035
595  em  1035
596  em  1035
597  em  1035
598  em  1035
599  em  1035
600  em  1035
601  em  1035
602  em  1035
603  em  1035
604  em  1035
605  em  1035
606  em  1035
607  em  1035
608  em  1035
609  em  1035
610  em  1035
611  em  1035
612  em  1035
613  em  1035
614  em  1035
615  em  1035
616  em  1035
617  em  1035
618  em  1035
619  em  1035
620  em  1035
621  em  1035
622  em  1035
623  em  1035
624  em  1035
625  em  1035
626  em  1035
627  em  1035
628  em  1035
629  em  1035
630  em  1035
631  em  1035
632  em  1035
633  em  1035
634  em  1035
635  em  1035
636  em  1035
637  em  1035
638  em  1035
639  em  1035
640  em  1035
641  em  1035
642  em  1035
643  em  1035
644  em  1035
645  em  1035
646  em  1035
647  em  1035
648  em  1035
649  em  1035
650  em  1035
651  em  1035
652  em  1035
653  em  1035
654  em  1035
655  em  1035
656  em  1035
657  em  1035
658  em  1035
659  em  1035
660  em  1035
661  em  1035
662  em  1035
663  em  1035
664  em  1035
665  e

In [211]:
df.head(200)

,resultId,raceId,driverId,constructorId,gridStart,position,positionText,positionOrder,pointsGained,laps,...,startsFrontRow,driverStandingsId,driverStandingsPoints,driverStandingsPosition,driverStandingsWins,constructorStandingsId,constructorStandingsPoints,constructorStandingsPosition,constructorStandingsWins,racesWon
0,1,18,1,1,1,1,1,1,10.0,58,...,True,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4.0
1,27,19,1,1,9,5,5,5,4.0,56,...,False,9.0,10.0,1.0,1.0,7.0,14.0,1.0,1.0,5.0
2,57,20,1,1,3,13,13,13,0.0,56,...,False,27.0,14.0,1.0,1.0,18.0,24.0,1.0,1.0,5.0
3,69,21,1,1,5,3,3,3,6.0,66,...,False,48.0,14.0,3.0,1.0,29.0,28.0,3.0,1.0,5.0
4,90,22,1,1,3,2,2,2,8.0,58,...,False,69.0,20.0,2.0,1.0,40.0,34.0,3.0,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,23523,976,1,131,1,5,5,5,10.0,51,...,True,68304.0,129.0,2.0,3.0,26804.0,222.0,1.0,4.0,56.0
196,23545,977,1,131,8,4,4,4,12.0,71,...,False,68326.0,139.0,2.0,3.0,26814.0,250.0,1.0,4.0,56.0
197,23562,978,1,131,1,1,1,1,25.0,51,...,True,68348.0,151.0,2.0,3.0,26824.0,287.0,1.0,5.0,56.0
198,23585,979,1,131,4,4,4,4,12.0,70,...,False,68370.0,176.0,2.0,4.0,26834.0,330.0,1.0,6.0,57.0
